# A Landau.jl tutorial

This notebook accompanies the paper 

[LD] Sebastian Mizera and Simon Telen, *Landau Discriminants*, arXiv: .

It illustrates the use of the Julia package Landau.jl for computing Landau discriminants. We start by adding the directory to the path and activating the environment. The latter makes sure that Julia uses the correct package versions for all auxiliary packages. 

In [7]:
push!(LOAD_PATH,string(pwd() * "/src"))
using Pkg; Pkg.activate()

  Activating project at `C:\Users\tristan\.julia\environments\v1.10`


We are now ready to import the package Landau.jl. This may take a minute.

In [11]:
using Landau

In Landau.jl, a Feynman diagram is represented by a list of edges and a list of nodes. An edge between vertices $i$ and $j$ is represented by a 2-tuple $[i,j]$. The list of nodes indicates to which vertices the external legs are attached. For instance, for the parachute diagram in [LD, Fig. 1], the four external legs are attached to vertex 1, 1, 2 and 3 respectively. 

In [12]:
edges = [[3,1],[1,2],[2,3],[2,3]];
nodes = [1,1,2,3];

We can compute the Symanzik polynomials ${\cal U}_G$ and ${\cal F}_G$ for this diagram as follows. 

In [13]:
F, U, α, p, mm = getF(edges, nodes);
println("F = $F")
println("--------------")
println("U = $U")

F = -(α₁*α₃*α₄ + α₃*α₂*α₄)*p₃₋₄ - (α₁*α₂*α₄ + α₁*α₃*α₂ + α₁*α₃*α₄)*p₁₋₄ - (α₁*α₂*α₄ + α₁*α₃*α₂ + α₁*α₃*α₄)*p₂₋₄ - (α₁*α₂*α₄ + α₁*α₃*α₂ + α₃*α₂*α₄)*p₁₋₃ - (α₁*α₂*α₄ + α₁*α₃*α₂ + α₃*α₂*α₄)*p₂₋₃ - (α₁*mm₁ + α₂*mm₂ + α₃*mm₃ + α₄*mm₄)*(α₁*α₃ + α₁*α₄ + α₂*α₄ + α₃*α₂ + α₃*α₄)
--------------
U = α₁*α₃ + α₁*α₄ + α₂*α₄ + α₃*α₂ + α₃*α₄


The matrix p contains $p_i \cdot p_j$ in its $(i,j)$ entry, where $p_i$ is the momentum vector of the $i$-th particle. To substitute these by the masses and Mandelstam invariants, we implemented the function substitute4legs for diagrams with 4 external particles. Analogously, for diagrams with 5 dangling edges there is a function substitute5legs. These functions also has the option to use equal masses for all internal and external particles.

In [14]:
F_genericmass, s, t, M, m = substitute4legs(F, p, mm);
println("F_genericmass = $(F_genericmass)")
println("-----------------------")
F_equalmass, s, t, M, m = substitute4legs(F, p, mm; equalM = true, equalm = true);
println("F_equalmass = $(F_equalmass)")

F_genericmass = (-1/2)*(-M₃ - M₄ + s)*(α₁*α₃*α₄ + α₃*α₂*α₄) + (-1/2)*(-M₂ - M₃ + t)*(α₁*α₂*α₄ + α₁*α₃*α₂ + α₃*α₂*α₄) + (-1/2)*(-M₁ - M₄ + t)*(α₁*α₂*α₄ + α₁*α₃*α₂ + α₁*α₃*α₄) + (-1/2)*(M₁ + M₃ - s - t)*(α₁*α₂*α₄ + α₁*α₃*α₂ + α₁*α₃*α₄) + (-1/2)*(M₂ + M₄ - s - t)*(α₁*α₂*α₄ + α₁*α₃*α₂ + α₃*α₂*α₄) - (m₁*α₁ + m₂*α₂ + m₃*α₃ + m₄*α₄)*(α₁*α₃ + α₁*α₄ + α₂*α₄ + α₃*α₂ + α₃*α₄)
-----------------------
F_equalmass = -(-M + (1/2)*s)*(α₁*α₃*α₄ + α₃*α₂*α₄) - (-M + (1/2)*t)*(α₁*α₂*α₄ + α₁*α₃*α₂ + α₁*α₃*α₄) - (-M + (1/2)*t)*(α₁*α₂*α₄ + α₁*α₃*α₂ + α₃*α₂*α₄) - (-M + (1/2)*(4*M - s - t))*(α₁*α₂*α₄ + α₁*α₃*α₂ + α₁*α₃*α₄) - (-M + (1/2)*(4*M - s - t))*(α₁*α₂*α₄ + α₁*α₃*α₂ + α₃*α₂*α₄) - (m*α₁ + m*α₂ + m*α₃ + m*α₄)*(α₁*α₃ + α₁*α₄ + α₂*α₄ + α₃*α₂ + α₃*α₄)


Note that the command getF also returns ${\cal U}_G$. This is computed as a byproduct since the first Symanzik polynomial appears in the definition of the second. If we want to compute just ${\cal U}_G$, we can use

In [15]:
U, M, α, vtcs = getU(edges);
println("U = $U")

U = α₁*α₃ + α₁*α₄ + α₂*α₄ + α₃*α₂ + α₃*α₄


Here's how to generate the critical point equations of ${\cal F}_G$. We load HomotopyContinuation.jl to represent these equations as a system.

In [16]:
LE, α, p, mm = LandauEquations(edges, nodes);
using HomotopyContinuation
System(LE)

System of length 4
 13 variables: mm₁, mm₂, mm₃, mm₄, p₁₋₃, p₂₋₃, p₁₋₄, p₂₋₄, p₃₋₄, α₁, α₂, α₃, α₄

 -mm₁*(α₁*α₃ + α₁*α₄ + α₂*α₄ + α₃*α₂ + α₃*α₄) - p₁₋₄*(α₂*α₄ + α₃*α₂ + α₃*α₄) - p₂₋₄*(α₂*α₄ + α₃*α₂ + α₃*α₄) - (α₃ + α₄)*(α₁*mm₁ + α₂*mm₂ + α₃*mm₃ + α₄*mm₄) - (α₂*α₄ + α₃*α₂)*p₁₋₃ - (α₂*α₄ + α₃*α₂)*p₂₋₃ - α₃*α₄*p₃₋₄
 -mm₂*(α₁*α₃ + α₁*α₄ + α₂*α₄ + α₃*α₂ + α₃*α₄) - p₁₋₃*(α₁*α₃ + α₁*α₄ + α₃*α₄) - p₂₋₃*(α₁*α₃ + α₁*α₄ + α₃*α₄) - (α₃ + α₄)*(α₁*mm₁ + α₂*mm₂ + α₃*mm₃ + α₄*mm₄) - (α₁*α₃ + α₁*α₄)*p₁₋₄ - (α₁*α₃ + α₁*α₄)*p₂₋₄ - α₃*α₄*p₃₋₄
 -mm₃*(α₁*α₃ + α₁*α₄ + α₂*α₄ + α₃*α₂ + α₃*α₄) - (α₁*α₂ + α₁*α₄)*p₁₋₄ - (α₁*α₂ + α₁*α₄)*p₂₋₄ - (α₁*α₂ + α₂*α₄)*p₁₋₃ - (α₁*α₂ + α₂*α₄)*p₂₋₃ - (α₁*α₄ + α₂*α₄)*p₃₋₄ - (α₁*mm₁ + α₂*mm₂ + α₃*mm₃ + α₄*mm₄)*(α₁ + α₂ + α₄)
 -mm₄*(α₁*α₃ + α₁*α₄ + α₂*α₄ + α₃*α₂ + α₃*α₄) - (α₁*α₂ + α₁*α₃)*p₁₋₄ - (α₁*α₂ + α₁*α₃)*p₂₋₄ - (α₁*α₂ + α₃*α₂)*p₁₋₃ - (α₁*α₂ + α₃*α₂)*p₂₋₃ - (α₁*α₃ + α₃*α₂)*p₃₋₄ - (α₁*mm₁ + α₂*mm₂ + α₃*mm₃ + α₄*mm₄)*(α₁ + α₂ + α₃)

The equations we will eventually solve are an affine version of these equaitons, see [LD, Sec. 3.2]. These affine equations can be computed by the function affineLandauEquations. Notice below that they only contain 3 out of 4 Schwinger parameters ($\alpha_4$ is set to 1) and the last equation is of the form $yg - 1 = 0$. 

In [17]:
LE, y, α, p, mm = affineLandauEquations(edges,nodes);
LE, s, t, M, m = substitute4legs(LE, p, mm); 
println("variables: $(variables(LE))")
println("last eq.: $(LE[end])")

variables: Variable[M₁, M₂, M₃, M₄, m₁, m₂, m₃, m₄, s, t, y, α₁, α₂, α₃]
last eq.: -1 + y*α₁*α₃*α₂*(1.0*α₁ + 1.0*α₂ + 1.0*α₃ + α₁*α₃ + α₃*α₂)


The projection of the variety defined by these 4 equations in 14 unknowns onto the space of parameters $s,t,M_i,m_e$ is dense in the Landau discriminant. We can compute the degree of the closure of this projection as follows.  

In [18]:
dproj = degreeProjection(LE, [α[1:end-1];y], [s;t;M;m]);
println("degree = $dproj")

Tracking 24 paths...   8%|███                           |  ETA: 0:01:52
  # paths tracked:                  2
  # non-singular solutions (real):  1 (0)
  # singular endpoints (real):      0 (0)






Tracking 24 paths... 100%|██████████████████████████████| Time: 0:00:10
  # paths tracked:                  24
  # non-singular solutions (real):  6 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         6 (0)


degree = 6


The number 6 can be found in [LD, Table 1]. We can also compute samples on the Landau discriminant using the command sampleProjection, see [LD, Sec. 3.2]. The optional input npoints is a lower bound on the number of samples you would like to compute. The default is 200. 

In [19]:
samples, monres, AA, BB, H, singsamples = sampleProjection(LE,[α[1:end-1];y],[s;t;M;m]; npoints = 100);
println("Computed $(length(samples)) samples")

Tracking 6 paths...  33%|███████████                    |  ETA: 0:00:07
  # paths tracked:                  2
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)






Tracking 6 paths... 100%|███████████████████████████████| Time: 0:00:03
  # paths tracked:                  6
  # non-singular solutions (real):  6 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         6 (0)


Computed 102 samples


We now repeat the steps above, restricting to equal masses for internal and external particles. In degreeProjection and sampleProjection, we use the option findSingular = true, as it turns out that this is an example where the incidence scheme has a non-reduced component (see [LD, Remark 5]).

In [20]:
LE, y, α, p, mm = affineLandauEquations(edges,nodes);
LE, s, t, M, m = substitute4legs(LE, p, mm; equalM = true, equalm = true);
dproj = degreeProjection(LE,[α[1:end-1];y], [s;t;M;m]; findSingular = true);
samples, monres, AA, BB, H, singsamples = sampleProjection(LE,[α[1:end-1];y],[s;t;M;m]; npoints = 100, findSingular = true);
println("-----------")
println("degree = $dproj")
println("Computed $(length(samples)) samples")
println("Computed $(length(singsamples)) singular samples")

Tracking 24 paths...   8%|███                           |  ETA: 0:00:41
  # paths tracked:                  2
  # non-singular solutions (real):  1 (0)
  # singular endpoints (real):      0 (0)






Tracking 24 paths...  88%|███████████████████████████   |  ETA: 0:00:01
  # paths tracked:                  21
  # non-singular solutions (real):  1 (0)
  # singular endpoints (real):      5 (0)






Tracking 24 paths... 100%|██████████████████████████████| Time: 0:00:03
  # paths tracked:                  24



  # singular endpoints (real):      6 (0)
  # total solutions (real):         8 (0)


found some singular solutions
singular component is estimated to have degree >= 1


found some singular solutions
singular component is estimated to have degree >= 1


-----------
degree = 3
Computed 68 samples
Computed 34 singular samples


Both degreeProjection and sampleProjection warn that some singular solutions were found. The degree of the discriminant is 3, and from the ratio between regular and singular samples we see that it is a union of surfaces of degree 2 and 1. In particular, it is reducible. To find the equation for the degree 2 surface, we interpolate the regular samples.

In [21]:
pol, N, gap = interpolate_deg(samples,2,[s;t;M;m]);
pol

(1.0 + -0.0*im)*M*m + (-3.27419185147561e-18 - 5.09919405619472e-16*im)*M*s + (-1.93576802126058e-16 - 8.82834067491234e-16*im)*M*t + (-0.4 - 3.70851482050028e-16*im)*m*s + (1.02134576245855e-15 + 1.58990085709072e-15*im)*m*t + (6.44123721821013e-16 + 4.94067441337399e-16*im)*s*t + (-0.0999999999999999 + 1.09710230106467e-15*im)*M^2 + (-0.899999999999999 - 3.09042901708356e-15*im)*m^2 + (-7.68442166148151e-17 - 1.32778158458167e-16*im)*t^2

This is a polynomial with complex floating point coefficients, whose imaginary part is negligible. We convert this into a polynomial with rational coefficients via the function rat in Landau.jl.

In [22]:
rat(pol)

M*m + (-2/5)*m*s + (-1/10)*M^2 + (-9/10)*m^2

Similarly, for the singular samples: 

In [23]:
pol, N, gap = interpolate_deg(singsamples,1,[s;t;M;m]);
rat(pol)

-M + m

The product of these two polynomials (up to a scalar factor) is found when interpolating all samples simultaneously. 

In [24]:
pol, N, gap = interpolate_deg([samples;singsamples],3,[s;t;M;m]);
rat(pol)

M*m^2 + (-11/19)*M^2*m + (-4/19)*m^2*s + (4/19)*M*m*s + (1/19)*M^3 + (-9/19)*m^3

The same result is computed with the blackbox function getLandauDiscriminant, which computes the discriminants directly from the graph using the the steps illustrated above. 

In [25]:
Δ, par = getLandauDiscriminant(edges,nodes);
println("--------------------")
println("the discriminant is:")
Δ[1]

-------------------------------------------
Slicing the incidence variety...


Tracking 24 paths...   8%|███                           |  ETA: 0:00:40
  # paths tracked:                  2
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      2 (0)






Tracking 24 paths... 100%|██████████████████████████████| Time: 0:00:03
  # paths tracked:                  24
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      6 (0)
  # total solutions (real):         8 (0)


found some singular solutions
singular component is estimated to have degree >= 1


found some singular solutions
singular component is estimated to have degree >= 1


found some singular solutions
singular component is estimated to have degree >= 1


Interpolating 24 samples ...


M*m^2 + (-11/19)*M^2*m + (-4/19)*m^2*s + (4/19)*M*m*s + (1/19)*M^3 + (-9/19)*m^3
--------------------
the discriminant is:


M*m^2 + (-11/19)*M^2*m + (-4/19)*m^2*s + (4/19)*M*m*s + (1/19)*M^3 + (-9/19)*m^3

If possible, this function returns the individual components of the discriminants. Here is the example of the double box (dbox) diagram. 


In [29]:
edges = [[1,2],[2,3],[3,4],[4,5],[5,6],[6,1],[3,6]]
nodes = [1,2,4,5]
Δ, par = getLandauDiscriminant(edges,nodes);

-------------------------------------------
Slicing the incidence variety...


Computing mixed cells... 95    Time: 0:00:00



Computing mixed cells... 201    Time: 0:00:00
  mixed_volume:  436


Tracking 436 paths...   0%|█                            |  ETA: 0:32:06
  # paths tracked:                  2
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      0 (0)






Tracking 436 paths...   4%|██                           |  ETA: 0:03:45
  # paths tracked:                  17
  # non-singular solutions (real):  1 (0)
  # singular endpoints (real):      0 (0)


...





Tracking 436 paths...  97%|█████████████████████████████|  ETA: 0:00:00
  # paths tracked:                  422
  # non-singular solutions (real):  8 (0)
  # singular endpoints (real):      0 (0)






Tracking 436 paths... 100%|█████████████████████████████| Time: 0:00:08
  # paths tracked:                  436
  # non-singular solutions (real):  8 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         8 (0)


Components have degrees [2, 4], total estimated degree is 6.
-------------------------------------------
Sampling component 1 ...


Interpolating 12 samples ...
M*m + (-1/4)*m*s - m*t + (1/4)*s*t
-------------------------------------------
Sampling component 2 ...


Interpolating 44 samples ...
M^2*m^2 + (-2/3)*M^3*m + (1/16)*m^2*s^2 + (1/9)*m^2*t^2 + (1/144)*s^2*t^2 + (-1/2)*M*m^2*s + (-2/3)*M*m^2*t + (1/6)*M^2*m*s + (1/9)*M^2*m*t + (-1/18)*M^2*s*t + (-1/18)*m*s*t^2 + (-5/72)*m*s^2*t + (1/6)*m^2*s*t + (5/18)*M*m*s*t + (1/9)*M^4


In [30]:
Δ[1]

M*m + (-1/4)*m*s - m*t + (1/4)*s*t

In [31]:
Δ[2]

M^2*m^2 + (-2/3)*M^3*m + (1/16)*m^2*s^2 + (1/9)*m^2*t^2 + (1/144)*s^2*t^2 + (-1/2)*M*m^2*s + (-2/3)*M*m^2*t + (1/6)*M^2*m*s + (1/9)*M^2*m*t + (-1/18)*M^2*s*t + (-1/18)*m*s*t^2 + (-5/72)*m*s^2*t + (1/6)*m^2*s*t + (5/18)*M*m*s*t + (1/9)*M^4

Finally, we show how to compute the number of master integrals of the double box diagram. The function $\chi$ implements exactly the code snippet shown in [LD, Sec. 5.2].

In [32]:
Landau.χ(edges,nodes)

Solutions found: 199    Time: 0:00:00
  tracked loops (queued):            288 (110)
  solutions in current (last) loop:  189 (9)





Solutions found: 227    Time: 0:00:01
  tracked loops (queued):            449 (5)
  solutions in current (last) loop:  217 (9)





Solutions found: 227    Time: 0:00:01
  tracked loops (queued):            540 (141)
  solutions in current (last) loop:  0 (217)





Solutions found: 227    Time: 0:00:01
  tracked loops (queued):            649 (32)
  solutions in current (last) loop:  0 (217)





Solutions found: 227    Time: 0:00:02
  tracked loops (queued):            784 (124)
  solutions in current (last) loop:  0 (0)


...




Solutions found: 227    Time: 0:00:05
  tracked loops (queued):            1394 (188)
  solutions in current (last) loop:  0 (0)





Solutions found: 227    Time: 0:00:06
  tracked loops (queued):            1582 (0)
  solutions in current (last) loop:  0 (0)
  generated loops (no change):       7 (5)


Certifying 227 solutions...   1%|█                      |  ETA: 0:02:12
  # processed:         2
  # certified (real):  2 (0)
  # distinct (real):   2 (0)




Certifying 227 solutions... 100%|███████████████████████| Time: 0:00:01
  # processed:         227
  # certified (real):  227 (0)
  # distinct (real):   227 (0)


227

The number 227 can be found in [LD, Tab. 3].